1. Get the Basin parameters per run
2. Get the DSS results. 
3. Filter the result. Retain only the dates with observed streamflow data
4. Compute the mean of observed data
5. compute the Observed - Predicted
6. Compute the NSE for the run

In [1]:
import os, datetime
import numpy as np
import pandas as pd
from pydsstools.heclib.dss import HecDss

In [2]:
# User defined inputs

# HMS File Paths
hms_project_directory = r"D:\AMH Philippines, Inc\EDC Projects - 20 NP24.XXX Amacan Water Balance HH\06 WORK FILES\05 HEC HMS\hms_edc_amacan_rdtc_calib"
hms_basin_name = 'EDC_Basin'
hms_dss_name = 'edc_run_v1'

# Observed Streamflow Data File path
obs_streamflow_path = r''

# ---- Working File Paths ----
hms_basin_path = os.path.join(hms_project_directory, f'{hms_basin_name}.basin')
hms_dss_path = os.path.join(hms_project_directory, f'{hms_dss_name}.dss')
csv_save_path = os.path.join(hms_project_directory, 'NSE_computation', 'NSE.csv')

print(f'HMS Project Directory: {os.path.basename(hms_project_directory)}')
print(f'HMS Working Basin File: {os.path.basename(hms_basin_path)}')
print(f'HMS Working DSS File: {os.path.basename(hms_dss_path)}')
print(f'HMS CSV Save File: {os.path.basename(csv_save_path)}')

HMS Project Directory: hms_edc_amacan_rdtc_calib
HMS Working Basin File: EDC_Basin.basin
HMS Working DSS File: edc_run_v1.dss
HMS CSV Save File: NSE.csv


In [3]:
# Define Functions

def parse_subbasin_reach(filepath):
    """
    Parses a text file with Subbasin and Reach blocks into a pandas DataFrame.
    """
    records = []
    current = None
    in_code = False

    with open(filepath, 'r') as f:
        for raw in f:
            line = raw.strip()
            
            # Detect new block
            if line.startswith(('Subbasin:', 'Reach:')):
                if current:
                    records.append(current)
                kind, name = line.split(':', 1)
                current = {'type': kind, 'name': name.strip()}
                in_code = False
                continue
            
            # Toggle code block on/off
            if line == '```':
                in_code = not in_code
                continue
            
            # End of block
            if line == 'End:':
                if current:
                    records.append(current)
                current = None
                in_code = False
                continue
            
            # Within a block, parse any "Key: Value" lines
            if ':' in line and current is not None:
                key, val = line.split(':', 1)
                col = key.strip().lower().replace(' ', '_').replace('-', '_')
                current[col] = val.strip()

    df = pd.DataFrame(records)
    df = df[['type', 'name', 'area', 'downstream',
       'discretization', 'canopy', 'allow_simultaneous_precip_et',
       'plant_uptake_method', 'initial_canopy_storage_percent',
       'canopy_storage_capacity', 'crop_coefficient', 'surface', 'initial_surface_storage_percent',
       'surface_storage_capacity', 'surface_albedo',
       'lossrate', 'percent_impervious_area',
       'initial_soil_storage_percent', 'initial_gw1_storage_percent',
       'initial_gw2_storage_percent', 'soil_maximum_infiltration',
       'soil_storage_capacity', 'soil_tension_capacity',
       'soil_maximum_percolation', 'groundwater_1_storage_capacity',
       'groundwater_1_routing_coefficient',
       'groundwater_1_maximum_percolation',
       'groundwater_2_storage_capacity',
       'groundwater_2_routing_coefficient',
       'groundwater_2_maximum_percolation', 'transform', 'lag',
       'unitgraph_type', 'baseflow', 'recession_factor',
       'initial_baseflow', 'threshold_flow_to_peak_ratio', 'route',
       'initial_variable', 'channel_loss']]
    return df

def save_with_sequential_run_id(df, master_csv=csv_save_path):
    """
    Appends df to master_csv, tagging each row with a run_id of the form 'Run N',
    where N is one greater than the highest existing run number in master_csv.
    """
    if not os.path.exists(os.path.dirname(master_csv)):
        os.makedirs(os.path.dirname(master_csv))

    # 1) Determine the next run number
    if os.path.exists(master_csv):
        # Read just the run_id column (faster than whole file)
        existing = pd.read_csv(master_csv, usecols=["run_id"])
        # Extract numbers from strings like 'Run 3'
        nums = existing["run_id"].dropna().str.extract(r"Run\s+(\d+)", expand=False)
        nums = pd.to_numeric(nums, errors="coerce").dropna()
        next_run = int(nums.max()) + 1 if not nums.empty else 1
    else:
        next_run = 1

    run_id = f"Run {next_run}"
    
    # 2) Tag the DataFrame
    df = df.copy()
    df["run_id"] = run_id

    # 3) Append (or create) the master CSV
    header = not os.path.exists(master_csv)
    df.to_csv(master_csv, mode="a", index=False, header=header)

    return run_id

In [83]:
# Read Basin file then store the variables in a DataFrame
df = parse_subbasin_reach(hms_basin_path)
save_with_sequential_run_id(df=df)


'Run 1'

In [28]:
xl_path = r"D:\AMH Philippines, Inc\EDC Projects - 20 NP24.XXX Amacan Water Balance HH\06 WORK FILES\05 HEC HMS\hms_edc_amacan_rdtc_calib\NSE_computation\observed_data.xlsx"

df_obs = pd.read_excel(xl_path)
df_obs

,date,extraction_point,discharge
0,2025-02-03,Masara River,0.518018
1,2025-02-07,Masara River,0.513433
2,2025-02-10,Masara River,0.512039
3,2025-02-12,Masara River,0.506568
4,2025-02-14,Masara River,0.531326
5,2025-02-17,Masara River,0.591109
6,2025-02-19,Masara River,0.705351
7,2025-03-03,Masara River,0.613489
8,2025-03-05,Masara River,0.641001
9,2025-03-07,Masara River,0.652466


In [41]:
# Read from DSS File
dss_pathname = '//Bunlang EP/FLOW/01Jan2025/1Day/RUN:edc_run_v1/'
startDate = '02Feb2025 00:00:00'
endDate = '08Apr2025 00:00:00'

with HecDss.Open(hms_dss_path) as dss:
    ts = dss.read_ts(dss_pathname, 
                     window=(startDate, endDate),
                     trim_missing=True)
    times = np.array(ts.pytimes)
    values = ts.values

    df_test = pd.DataFrame(zip(times, values), columns=['date', 'pred_value'])
    
    allowed = [
        "03/02/2025","07/02/2025","10/02/2025","12/02/2025","14/02/2025","17/02/2025",
        "19/02/2025","03/03/2025","05/03/2025","07/03/2025","12/03/2025","17/03/2025",
        "24/03/2025","26/03/2025","28/03/2025","31/03/2025","02/04/2025","07/04/2025"
    ]

    allowed = pd.to_datetime(allowed, dayfirst=True)

    mask = df_test['date'].isin(allowed)
    df_filtered = df_test[mask].reset_index(drop=True)

# Merge DataFrame

# Filter observed data - retain Bulang Creek
df_obs_bunlang = df_obs[df_obs['extraction_point'] == 'Bunlang Creek']

# Merged two DataFrames
df_merged_bunlang = pd.merge(df_filtered, 
                             df_obs_bunlang,
                             on=['date'])

# NSE Computation
mean_observed = df_merged_bunlang['discharge'].mean()

df_merged_bunlang['nse_numerator'] = np.power(df_merged_bunlang['discharge'] - df_merged_bunlang['pred_value'], 2)
df_merged_bunlang['nse_denominator'] = np.power(df_merged_bunlang['discharge'] - mean_observed, 2)

nse = 1.0 - (df_merged_bunlang['nse_numerator'].sum() / df_merged_bunlang['nse_denominator'].sum())